In [1]:
import pandas as pd

df = pd.read_csv('../Data/2019.csv', encoding='utf-8-sig')
df.tail()

,date,press,category,headline,url,text,img
128646,2019/06/30,세계일보,"정치,국제",[WT논평]The reason why we must denuclearize Nort...,http://www.segye.com/content/html/2019/06/30/2...,By Robert R. Monroe (former director of the De...,[]
128647,2019/06/30,세계일보,"국제,정치","[사설] 日 경제보복 카드 우려, 정부는 전략적 대응 모색하길",http://www.segye.com/content/html/2019/06/30/2...,일본이 강제징용 판결에 대해 경제보복 카드를 꺼내든다고 한다. 일본 정부가 TV·스...,['//img.segye.com/content/image/2019/06/30/201...
128648,2019/06/30,세계일보,정치,[사설] 84일 만의 국회 본회의… ‘완전한 정상화’ 서두를 때다,http://www.segye.com/content/html/2019/06/30/2...,국회가 지난 28일 자유한국당을 포함한 여야 5당이 모두 참석한 가운데 본회의를 열...,['//img.segye.com/content/image/2019/06/28/201...
128649,2019/06/30,세계일보,정치,"[사설] 남·북·미 정상 판문점 회동, 실질적 비핵화로 이어져야",http://www.segye.com/content/html/2019/06/30/2...,분단·냉전 상징서 역사적 만남 / 트럼프 “북·미 실무협상 할 것” / 北 비핵화 ...,['//img.segye.com/content/image/2019/06/30/201...
128650,2019/06/30,중앙일보,정치,"조국 ""사실상 종전선언""…靑참모들 일제히 ""역사적인 날""",https://news.joins.com/article/olink/23103851,"조국 청와대 민정수석이 30일 남북미 세 정상의 판문점 회동에 대해 ""세 지도자의 ...",['https://pds.joins.com/news/component/htmlpho...


In [2]:
from kiwipiepy import Kiwi
import pandas as pd
import math

kiwi = Kiwi()
kiwi.load_user_dictionary('sorted_wordlist_end.txt')
kiwi.prepare()

stopword = []
def rm_stopword(stopword_file): 
    files = open(stopword_file, 'r', encoding="utf-8")
    while True: #불용어 리스트 생성
        line = files.readline()
        if not line: break
        wordlist = line.split('\t')
        if wordlist[1].startswith('N'):
            stopword.append(wordlist[0])
    stopwords = set(stopword)
    files.close()
    return stopword

def npmi(a, b, n, z):
    return (math.log((n*z)/(a*b))) / (math.log(n/z))

def npmi3(a, b, c, n, z):
    a = math.log((n*n*z)/(a*b*c))
    b = 2*math.log(n/z)
    return a / b

In [16]:
from kss import split_sentences
from collections import Counter
import pprint as pp
import re

rm_stopword('한국어불용어100.txt')
docs = []
for news in open('News_Contents.txt', encoding='utf-8'):
    doc = []
    context = []
    com = []
    lines = split_sentences(news)

    #형태소 분석
    for res in kiwi.analyze(lines):
        content = [word for word, tag, _, _ in res[0][0] if tag.startswith('N') and word not in stopword] #불용어사전 적용
        context += content
        doc.append(content)

    #2 token npmi
    for line in doc:
        com += list(zip(line, line[1:]))
    nmi = {}
    n = len(context)
    cnt = Counter(context)
    z = Counter(com)
    for pair in com:
        pmi = npmi(cnt[pair[0]], cnt[pair[1]], n, z[pair])
        if -1 < pmi < 1:
            nmi[pair] = pmi
    result = sorted(nmi.items(), key = lambda item : item[1], reverse=True)
    merge_token = {key[0]+" "+key[1] for key, item in result if item > 0.89}
    print(merge_token)
    
    # #3 token npmi 
    # com3 = []
    # for line in doc:
    #     com3 += zip(line, line[1:], line[2:])
    # nmi3 = {}
    # cnt = Counter(context)
    # z = Counter(com3)
    # for pair in com3:
    #     pmi = npmi3(cnt[pair[0]], cnt[pair[1]], cnt[pair[2]], n, z[pair])
    #     if -1 < pmi < 1:
    #         nmi3[pair] = pmi
    # result = sorted(nmi3.items(), key = lambda item : item[1], reverse=True)
    # print(result)

    #2 token N_GRAM 연결
    # bi_gram = {}
    # tmp = []
    # for line in doc:
    #     tmp += list(zip(line, line[1:]))
    # for word in tmp:
    #     string = word[0]+'_'+word[1]
    #     if string in bi_gram:
    #         bi_gram[string] += 1
    #     else:
    #         bi_gram[string] = 1
    
    # bi_gram = sorted(bi_gram.items(), key = lambda item : item[1], reverse=True)
    # merge_token = {key for key, item in bi_gram if item > 2}
    # print(merge_token)

    #3 token N_GRAM 연결
    # word_list = []
    # tmp3 = []
    # for line in doc:
    #     tmp3 += zip(line, line[1:], line[2:])
    # for word in tmp3:
    #     word_list.append('_'.join(list(word)))
    # tri_gram = Counter(word_list)
    # tri_gram = sorted(tri_gram.items(), key = lambda item : item[1], reverse=True)
    # merge_token = {key for key, item in tri_gram if item > 2}
    # print(merge_token)

    for token in merge_token:
        for content in doc:
            for i in range(len(content)-1):
                cmp_str = '_'.join(content[i:i+2])
                if cmp_str == token:
                    content.pop(i)
                    content.pop(i)
                    content.insert(i, token)
    docs.append(doc)
    print("===================================")
    if len(docs) > 100:
        break
    # if len(docs) % 10000 == 0:
    #     print(len(docs))

# df['N_list'] = docs
# df.to_csv('2019_pre.csv', encoding='utf-8-sig')

{'문 의장'}
{'포함 정규', '신규 직원'}
{'문 대통령'}
{'소비 증진', '축산 바이어', '인하 동결'}
{'국회 페 미'}
{'업무 추진'}
{'유해 물질', '실내 공기'}
{'신흥 국', '성장세 둔화', '국제 자본', '브렉 시트'}
set()
{'김 대표'}
{'자산 관리'}
{'최저 임금'}
{'인 이하', '사산 휴가', '대상 기업', '통상 임금'}
{'사무관 주장', '차관 보고', '국가 채무'}
{'기회 균등'}
set()
{'지하 도로', '퇴근 시간대'}
{'스포츠 아웃', '동물 학대', '가격 책정'}
{'후 우선'}
{'잔재 청산'}
{'문단 압구정', '바이오 업종', '임 KB국민은행', '예 적금', '정우성 신한은행', '신한 분당', '무역 전쟁'}
{'위험 관리'}
set()
set()
{'중도 상환', '이자 부담', '상승 위험', '상환 수수료'}
set()
{'일본군 위안부', '잔재 청산'}
{'공유 개념', '총생산 달러', '패망 전망', '우대 조항', '운동 발발', '파리 샤를', '직전 수정자본주의', '직전 충칭', '승리 조지', '의무 규정', '그 이름', '체제 도입', '공항 이스라엘', '수준 지적', '발전 계승', '긍정 평가', '남북 관계', '드골 공항', '가담 진보', '해방 직전', '경제 대국', '주년 기념비', '영역 개입', '개선 통일', '모습 확인', '보장 시기', '내용 선언', '요구 권리', '자리 조지', '건국 강령', '민주 공화', '통일 무드', '평등 자유', '운동 직후', '혁명 뒤', '폴란드 안', '서울대 사회학', '산물 설명', '구 체제', '체제 완벽', '사회학 과', '개념 발전', '현실 개탄', '민주 공화정', '기획 민족주의', '계승 필요', '출범 해체', '모습 누구', '무엇 남북', '선언 수준', '국제 보장', '주석 대통령', '대 경제', '카이로 선언', '초기 순수', '임정은 설립

In [109]:
import tomotopy as tp

def extract_topic(doc):
    model = tp.LDAModel(k=5, alpha=0.1, eta=0.001, min_cf=3,min_df=1, tw=tp.TermWeight.PMI)
    for line in doc:
        model.add_doc(line) #추출하고 모델안의 문헌을 넣는다. 즉, 학습과정에 쓰일 문헌을 생성
    model.train(0) #학습 초기화
    s = []
    for j in range(0,100):
        model.train(10) #문헌 학습, 안에 숫자는 깁스 샘플링의 반복횟수
		#이때 기본값으로 시스템내 가용한 모든 스레드의 개수사용, 그리고 병렬화 방법을 찾아서 실행시켜준다
    for k in range(model.k): #k는 토픽의 개수
        res = model.get_topic_words(k, top_n=10) #하위 토픽 i에 해당하는 top_n개의 단어 반환
        print(k,res)
        for w, p in res:
            if w not in s.keys():
                s.append(w)

        # s = ', '.join(w for w, p in res)
    # print(s)
    return ', '.join(s)

In [110]:
for doc in docs:
    print(extract_topic(doc))

0 [('해', 0.6629379987716675), ('국회', 0.33660975098609924), ('의장', 0.00022607484424952418), ('문', 0.00022607484424952418)]


AttributeError: 'list' object has no attribute 'keys'